# Guess That Number - Exploratory Data Analysis

## Set Up Environment

In [1]:
import sqlite3
import pandas as pd
import numpy as np

In [2]:
conn = sqlite3.connect('guess_that_number.db')
c = conn.cursor()

happy_path_query = """
SELECT
    s.id session_id,
    s.time app_start_time,
    gm.game_id,
    gm.level_of_difficulty_type_id,
    l.code level_of_difficulty_type,
    gm.range_low low_range,
    gm.range_high high_range,
    gm.winning_number,
    gm.time game_start_time,
    gm.error range_error,
    gs.guess_id,
    gs.hint_type_id,
    h.code hint_type,
    gs.hint,
    gs.hint_number,
    gs.time guess_entry_time,
    gs.guess,
    gs.feedback,
    gs.error guess_error,
    o.outcome_id,
    o.outcome_type_id,
    ot.code outcome_type,
    o.score,
    o.feedback_type,
    o.improvement_area_id,
    o.recommendation_type,
    o.time game_end_time,
    o.play_again
FROM session s
    LEFT JOIN game gm ON s.id = gm.session_id
    LEFT JOIN guess gs ON gm.game_id = gs.game_id
    LEFT JOIN outcome o ON gm.game_id = o.game_id
    LEFT JOIN level_of_difficulty_type l ON gm.level_of_difficulty_type_id = l.id
    LEFT JOIN hint_type h ON gs.hint_type_id = h.id
    LEFT JOIN outcome_type ot ON o.outcome_type_id = ot.id
ORDER BY session_id, gm.game_id, gs.guess_id
"""

range_error_query = """
SELECT
    s.id session_id,
    s.time app_start_time,
    gm.game_id,
    gm.time game_start_time,
    gm.error_type_id,
    e.code range_error_type
FROM session s
    LEFT JOIN game gm ON s.id = gm.session_id
    JOIN error_type e ON gm.error_type_id = e.id
WHERE gm.error = 1"""

guess_error_query = """
SELECT
    s.id session_id,
    s.time app_start_time,
    gm.game_id,
    gm.level_of_difficulty_type_id,
    l.code level_of_difficulty_type,
    gm.range_low low_range,
    gm.range_high high_range,
    gm.winning_number,
    gm.time game_start_time,
    gs.guess_id,
    gs.time guess_entry_time,
    gs.guess,
    gs.error_type_id,
    e.code guess_error_type
FROM session s
    LEFT JOIN game gm ON s.id = gm.session_id
    LEFT JOIN guess gs ON gm.game_id = gs.game_id
    LEFT JOIN level_of_difficulty_type l ON gm.level_of_difficulty_type_id = l.id
    JOIN error_type e ON gs.error_type_id = e.id
WHERE gs.error = 1"""

game_data = c.execute(happy_path_query).fetchall()
range_error_data = c.execute(range_error_query).fetchall()
guess_error_data = c.execute(guess_error_query).fetchall()

conn.commit()
conn.close()

In [3]:
game_data_cols = [
    "session_id",
    "app_start_time",
    "game_id",
    "level_of_difficulty_type_id",
    "level_of_difficulty_type",
    "low_range",
    "high_range",
    "winning_number",
    "game_start_time",
    "range_error",
    "guess_id",
    "hint_type_id",
    "hint_type",
    "hint",
    "hint_number",
    "guess_entry_time",
    "guess",
    "feedback",
    "guess_error",
    "outcome_id",
    "outcome_type_id",
    "outcome_type",
    "score",
    "feedback_type",
    "improvement_area_id",
    "recommendation_type",
    "game_end_time",
    "play_again"
]
game_data = pd.DataFrame(game_data, columns=game_data_cols)

In [4]:
range_error_cols = [
    "session_id",
    "app_start_time",
    "game_id",
    "game_start_time",
    "range_error_type_id",
    "range_error_type"
]
range_error_data = pd.DataFrame(range_error_data, columns=range_error_cols)

In [5]:
guess_error_cols = [
    "session_id",
    "app_start_time",
    "game_id",
    "level_of_difficulty_type_id",
    "level_of_difficulty_type",
    "low_range",
    "high_range",
    "winning_number",
    "game_start_time",
    "guess_id",
    "guess_entry_time",
    "guess",
    "guess_error_type_id",
    "guess_error_type"
]
guess_error_data = pd.DataFrame(guess_error_data, columns=guess_error_cols)

In [6]:
game_data.insert(10, "range_error_type_id", game_data.game_id.map(range_error_data.set_index("game_id").range_error_type_id))
game_data.insert(11, "range_error_type", game_data.game_id.map(range_error_data.set_index("game_id").range_error_type))

In [7]:
game_data.insert(21, "guess_error_type_id", game_data.guess_id.map(guess_error_data.set_index("guess_id").guess_error_type_id))
game_data.insert(22, "guess_error_type", game_data.guess_id.map(guess_error_data.set_index("guess_id").guess_error_type))

In [8]:
game_data.head(3)

,session_id,app_start_time,game_id,level_of_difficulty_type_id,level_of_difficulty_type,low_range,high_range,winning_number,game_start_time,range_error,...,guess_error_type,outcome_id,outcome_type_id,outcome_type,score,feedback_type,improvement_area_id,recommendation_type,game_end_time,play_again
0,1,2021-06-22 15:14:16,1.0,1.0,easy,1,10,7.0,2021-06-22 15:14:20,0.0,...,non_integer,1.0,1.0,win,80.0,None,NaN,None,2021-06-22 15:14:42,1.0
1,1,2021-06-22 15:14:16,1.0,1.0,easy,1,10,7.0,2021-06-22 15:14:20,0.0,...,NaN,1.0,1.0,win,80.0,None,NaN,None,2021-06-22 15:14:42,1.0
2,1,2021-06-22 15:14:16,1.0,1.0,easy,1,10,7.0,2021-06-22 15:14:20,0.0,...,out_of_range,1.0,1.0,win,80.0,None,NaN,None,2021-06-22 15:14:42,1.0


## Inspect and Clean Data

In [9]:
game_data.shape

(58, 32)

In [10]:
game_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 32 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   session_id                   58 non-null     int64  
 1   app_start_time               58 non-null     object 
 2   game_id                      54 non-null     float64
 3   level_of_difficulty_type_id  51 non-null     float64
 4   level_of_difficulty_type     51 non-null     object 
 5   low_range                    51 non-null     object 
 6   high_range                   51 non-null     object 
 7   winning_number               51 non-null     float64
 8   game_start_time              54 non-null     object 
 9   range_error                  54 non-null     float64
 10  range_error_type_id          3 non-null      float64
 11  range_error_type             3 non-null      object 
 12  guess_id                     51 non-null     float64
 13  hint_type_id          

In [11]:
game_data.describe()

,session_id,game_id,level_of_difficulty_type_id,winning_number,range_error,range_error_type_id,guess_id,hint_type_id,hint_number,guess_error,guess_error_type_id,outcome_id,outcome_type_id,score,improvement_area_id,play_again
count,58.000000,54.000000,51.000000,51.000000,54.000000,3.000000,51.000000,33.000000,33.000000,51.000000,2.000000,46.000000,46.000000,42.000000,4.0,46.000000
mean,6.103448,10.222222,2.196078,56.941176,0.055556,2.333333,26.000000,2.212121,1.696970,0.039216,4.500000,7.739130,1.086957,69.285714,1.0,0.456522
std,3.796072,5.745110,1.216875,138.933713,0.231212,1.154701,14.866069,1.317136,0.809508,0.196039,0.707107,4.239679,0.284885,17.860278,0.0,0.503610
min,1.000000,1.000000,1.000000,-49.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,4.000000,1.000000,1.000000,40.000000,1.0,0.000000
25%,4.000000,5.250000,1.000000,4.000000,0.000000,2.000000,13.500000,1.000000,1.000000,0.000000,4.250000,4.000000,1.000000,52.500000,1.0,0.000000
50%,5.000000,10.000000,2.000000,7.000000,0.000000,3.000000,26.000000,2.000000,2.000000,0.000000,4.500000,8.000000,1.000000,80.000000,1.0,0.000000
75%,8.000000,14.750000,3.000000,61.000000,0.000000,3.000000,38.500000,2.000000,2.000000,0.000000,4.750000,11.000000,1.000000,80.000000,1.0,1.000000
max,15.000000,20.000000,4.000000,586.000000,1.000000,3.000000,51.000000,6.000000,4.000000,1.000000,5.000000,15.000000,2.000000,100.000000,1.0,1.000000


In [12]:
game_data.describe(include='object')

,app_start_time,level_of_difficulty_type,low_range,high_range,game_start_time,range_error_type,hint_type,hint,guess_entry_time,guess,feedback,guess_error_type,outcome_type,feedback_type,recommendation_type,game_end_time
count,58,51,51,51,54,3,33,33,51,51,17,2,46,4,0,46
unique,15,4,4,6,20,2,6,29,51,36,2,2,2,1,0,15
top,2021-06-22 15:17:08,easy,1,10,2021-06-22 15:18:41,invalid,multiple,Nice try! Hint: It is a perfect square.,2021-06-22 15:26:55,1,good,non_integer,win,improvement,NaN,2021-06-22 15:19:52
freq,11,21,42,21,4,2,18,2,1,6,16,1,42,4,NaN,4


In [13]:
game_data['app_start_time'] = pd.to_datetime(game_data.app_start_time)
game_data['game_start_time'] = pd.to_datetime(game_data.game_start_time)
game_data['guess_entry_time'] = pd.to_datetime(game_data.guess_entry_time)
game_data['game_end_time'] = pd.to_datetime(game_data.game_end_time)

In [14]:
game_data.level_of_difficulty_type.value_counts()

easy      21
custom    12
medium    11
hard       7
Name: level_of_difficulty_type, dtype: int64

In [15]:
game_data.hint_type.value_counts()

multiple          18
factor             9
perfect_square     3
prime              1
even_odd           1
digit_sum          1
Name: hint_type, dtype: int64

In [16]:
game_data.outcome_type.value_counts()

win     42
lose     4
Name: outcome_type, dtype: int64

In [17]:
game_data.guess_error.value_counts()

0.0    49
1.0     2
Name: guess_error, dtype: int64

In [18]:
game_data.range_error.value_counts()

0.0    51
1.0     3
Name: range_error, dtype: int64

## Split Data Into Groups

In [19]:
no_games_cols = ["session_id", "app_start_time"]
no_games = game_data.loc[game_data.game_id.isna(), no_games_cols].copy()
no_games

,session_id,app_start_time
49,10,2021-06-22 15:27:41
50,11,2021-06-22 15:45:06
51,12,2021-06-22 15:45:46
52,13,2021-06-22 16:05:21


In [20]:
games = game_data[game_data.game_id.notna()].copy()

range_errors_cols = ["session_id", "app_start_time", "game_id", "game_start_time", "range_error", "range_error_type_id", 
                     "range_error_type"]
range_errors = games.loc[games.range_error == 1, range_errors_cols].copy()
range_errors

,session_id,app_start_time,game_id,game_start_time,range_error,range_error_type_id,range_error_type
13,4,2021-06-22 15:17:08,5.0,2021-06-22 15:17:20,1.0,3.0,invalid
14,4,2021-06-22 15:17:08,6.0,2021-06-22 15:17:25,1.0,1.0,comparison
43,8,2021-06-22 15:25:23,16.0,2021-06-22 15:26:09,1.0,3.0,invalid


In [21]:
games_started_cols = [x for x in list(games.columns) if x not in ["range_error", "range_error_type_id", "range_error_type"]]
games_started = games.loc[games.range_error == 0, games_started_cols].copy()
games_started = games_started[games_started.guess_id.notna()].copy()
games_started['low_range'] = games_started.low_range.astype('int')
games_started['high_range'] = games_started.high_range.astype('int')
games_started.insert(7, 'range_size', games_started.high_range - games_started.low_range + 1)
games_started

,session_id,app_start_time,game_id,level_of_difficulty_type_id,level_of_difficulty_type,low_range,high_range,range_size,winning_number,game_start_time,...,guess_error_type,outcome_id,outcome_type_id,outcome_type,score,feedback_type,improvement_area_id,recommendation_type,game_end_time,play_again
0,1,2021-06-22 15:14:16,1.0,1.0,easy,1,10,10,7.0,2021-06-22 15:14:20,...,non_integer,1.0,1.0,win,80.0,None,NaN,None,2021-06-22 15:14:42,1.0
1,1,2021-06-22 15:14:16,1.0,1.0,easy,1,10,10,7.0,2021-06-22 15:14:20,...,NaN,1.0,1.0,win,80.0,None,NaN,None,2021-06-22 15:14:42,1.0
2,1,2021-06-22 15:14:16,1.0,1.0,easy,1,10,10,7.0,2021-06-22 15:14:20,...,out_of_range,1.0,1.0,win,80.0,None,NaN,None,2021-06-22 15:14:42,1.0
3,1,2021-06-22 15:14:16,1.0,1.0,easy,1,10,10,7.0,2021-06-22 15:14:20,...,NaN,1.0,1.0,win,80.0,None,NaN,None,2021-06-22 15:14:42,1.0
4,1,2021-06-22 15:14:16,2.0,1.0,easy,1,10,10,1.0,2021-06-22 15:14:46,...,NaN,2.0,1.0,win,100.0,None,NaN,None,2021-06-22 15:14:51,0.0
5,2,2021-06-22 15:15:02,3.0,2.0,medium,1,100,100,92.0,2021-06-22 15:15:07,...,NaN,3.0,1.0,win,40.0,improvement,1.0,None,2021-06-22 15:15:32,0.0
6,2,2021-06-22 15:15:02,3.0,2.0,medium,1,100,100,92.0,2021-06-22 15:15:07,...,NaN,3.0,1.0,win,40.0,improvement,1.0,None,2021-06-22 15:15:32,0.0
7,2,2021-06-22 15:15:02,3.0,2.0,medium,1,100,100,92.0,2021-06-22 15:15:07,...,NaN,3.0,1.0,win,40.0,improvement,1.0,None,2021-06-22 15:15:32,0.0
8,2,2021-06-22 15:15:02,3.0,2.0,medium,1,100,100,92.0,2021-06-22 15:15:07,...,NaN,3.0,1.0,win,40.0,improvement,1.0,None,2021-06-22 15:15:32,0.0
9,3,2021-06-22 15:15:45,4.0,3.0,hard,1,1000,1000,93.0,2021-06-22 15:15:50,...,NaN,4.0,2.0,lose,NaN,None,NaN,None,2021-06-22 15:16:57,0.0


In [22]:
games_started.iloc[10]

session_id                                                                 3
app_start_time                                           2021-06-22 15:15:45
game_id                                                                  4.0
level_of_difficulty_type_id                                              3.0
level_of_difficulty_type                                                hard
low_range                                                                  1
high_range                                                              1000
range_size                                                              1000
winning_number                                                          93.0
game_start_time                                          2021-06-22 15:15:50
guess_id                                                                11.0
hint_type_id                                                             6.0
hint_type                                                          digit_sum

In [23]:
def calc_avg_guess_time(guess_id):
    game_id = games_started.loc[games_started.guess_id == guess_id, "game_id"].iloc[0]
    subset_df = games_started[games_started.game_id == game_id]
    guess_ids = list(subset_df.guess_id.unique())
    guess_ids.sort()
    guess_index = guess_ids.index(guess_id)
    
    guess_entry_time = subset_df.loc[subset_df.guess_id == guess_id, "guess_entry_time"].iloc[0]
    game_start_time = subset_df.loc[subset_df.guess_id == guess_id, "game_start_time"].iloc[0]
    game_time = (guess_entry_time - game_start_time).total_seconds()
    
    avg_guess_time = round(game_time / (guess_index + 1), 1)
    
    return avg_guess_time

In [24]:
games_started.insert(16, 'avg_guess_time', games_started.guess_id.apply(calc_avg_guess_time))

In [25]:
games_started.avg_guess_time.unique()

array([ 5. ,  5.5,  5.7,  4. ,  4.5,  7. ,  6.2,  6.5,  8.7, 11.2,  6. ,
        7.3,  3.8,  3.5,  3. ,  7.5,  8. , 17.8, 20.3, 27. , 14. ,  4.7])

In [26]:
games_summary_cols = ['session_id', 'app_start_time', 'level_of_difficulty_type_id', 'level_of_difficulty_type', 
                      'low_range', 'high_range', 'range_size', 'winning_number', 'game_start_time', 'outcome_id', 
                      'outcome_type_id', 'outcome_type', 'score', 'feedback_type', 'improvement_area_id', 
                      'recommendation_type', 'game_end_time', 'play_again']

games_summary = games_started.pivot_table(index='game_id', values='guess_id', aggfunc=len)
for col in games_summary_cols:
    games_summary[col] = games_summary.index.map(games_started.pivot_table(index='game_id', values=col, aggfunc=max, dropna=False)[col])

games_summary.rename(columns={'guess_id': 'total_guesses'}, inplace=True)
games_summary['total_hints'] = games_summary.index.map(games_started.pivot_table(index='game_id', values='hint_number', aggfunc=max, fill_value=0).hint_number)
games_summary['guess_errors'] = games_summary.index.map(games_started.pivot_table(index='game_id', values='guess_error', aggfunc=sum, fill_value=0).guess_error)

games_summary_col_order = [1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 19, 20, 10, 11, 12, 13, 14, 15, 16, 17, 18]
games_summary = games_summary.iloc[:, games_summary_col_order]

games_summary

,session_id,app_start_time,level_of_difficulty_type_id,level_of_difficulty_type,low_range,high_range,range_size,winning_number,game_start_time,total_guesses,...,guess_errors,outcome_id,outcome_type_id,outcome_type,score,feedback_type,improvement_area_id,recommendation_type,game_end_time,play_again
game_id,,,,,,,,,,,,,,,,,,,,,
1.0,1,2021-06-22 15:14:16,1.0,easy,1,10,10,7.0,2021-06-22 15:14:20,4.0,...,2,1.0,1.0,win,80.0,NaN,NaN,NaN,2021-06-22 15:14:42,1.0
2.0,1,2021-06-22 15:14:16,1.0,easy,1,10,10,1.0,2021-06-22 15:14:46,1.0,...,0,2.0,1.0,win,100.0,NaN,NaN,NaN,2021-06-22 15:14:51,0.0
3.0,2,2021-06-22 15:15:02,2.0,medium,1,100,100,92.0,2021-06-22 15:15:07,4.0,...,0,3.0,1.0,win,40.0,improvement,1.0,NaN,2021-06-22 15:15:32,0.0
4.0,3,2021-06-22 15:15:45,3.0,hard,1,1000,1000,93.0,2021-06-22 15:15:50,4.0,...,0,4.0,2.0,lose,NaN,NaN,NaN,NaN,2021-06-22 15:16:57,0.0
7.0,4,2021-06-22 15:17:08,4.0,custom,50,75,26,61.0,2021-06-22 15:17:29,3.0,...,0,5.0,1.0,win,80.0,NaN,NaN,NaN,2021-06-22 15:17:51,1.0
8.0,4,2021-06-22 15:17:08,1.0,easy,1,10,10,1.0,2021-06-22 15:18:02,4.0,...,0,6.0,1.0,win,70.0,NaN,NaN,NaN,2021-06-22 15:18:17,1.0
9.0,4,2021-06-22 15:17:08,1.0,easy,1,10,10,5.0,2021-06-22 15:18:20,2.0,...,0,7.0,1.0,win,90.0,NaN,NaN,NaN,2021-06-22 15:18:27,0.0
10.0,5,2021-06-22 15:18:38,2.0,medium,1,100,100,55.0,2021-06-22 15:18:41,4.0,...,0,8.0,1.0,win,40.0,NaN,NaN,NaN,2021-06-22 15:19:52,1.0
11.0,5,2021-06-22 15:18:38,2.0,medium,1,100,100,5.0,2021-06-22 15:19:58,3.0,...,0,9.0,1.0,win,60.0,NaN,NaN,NaN,2021-06-22 15:20:15,0.0


In [27]:
games_summary.total_guesses.value_counts()

3.0    8
4.0    5
2.0    3
1.0    1
Name: total_guesses, dtype: int64

In [28]:
games_started.insert(10, 'total_guesses', games_started.game_id.map(games_summary.total_guesses))
games_started.insert(11, 'total_hints', games_started.game_id.map(games_summary.total_hints))
games_started.insert(12, 'total_guess_errors', games_started.game_id.map(games_summary.guess_errors))

In [29]:
games_started.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51 entries, 0 to 57
Data columns (total 34 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   session_id                   51 non-null     int64         
 1   app_start_time               51 non-null     datetime64[ns]
 2   game_id                      51 non-null     float64       
 3   level_of_difficulty_type_id  51 non-null     float64       
 4   level_of_difficulty_type     51 non-null     object        
 5   low_range                    51 non-null     int32         
 6   high_range                   51 non-null     int32         
 7   range_size                   51 non-null     int32         
 8   winning_number               51 non-null     float64       
 9   game_start_time              51 non-null     datetime64[ns]
 10  total_guesses                51 non-null     float64       
 11  total_hints                  50 non-null     fl

In [30]:
games_not_finished = games_started.loc[games_started.outcome_id.isna(), :"guess_error"].copy()
games_not_finished

,session_id,app_start_time,game_id,level_of_difficulty_type_id,level_of_difficulty_type,low_range,high_range,range_size,winning_number,game_start_time,...,guess_id,hint_type_id,hint_type,hint,hint_number,guess_entry_time,avg_guess_time,guess,feedback,guess_error
53,14,2021-06-22 16:09:22,19.0,1.0,easy,1,10,10,5.0,2021-06-22 16:09:27,...,47.0,2.0,multiple,Nice try! Hint: 10 is a multiple.,1.0,2021-06-22 16:09:31,4.0,7,None,0.0
54,14,2021-06-22 16:09:22,19.0,1.0,easy,1,10,10,5.0,2021-06-22 16:09:27,...,48.0,NaN,None,None,NaN,2021-06-22 16:09:35,4.0,5,None,0.0
55,15,2021-06-22 16:20:59,20.0,1.0,easy,1,10,10,4.0,2021-06-22 16:21:03,...,49.0,5.0,perfect_square,Nice try! Hint: It is a perfect square.,1.0,2021-06-22 16:21:06,3.0,8,None,0.0
56,15,2021-06-22 16:20:59,20.0,1.0,easy,1,10,10,4.0,2021-06-22 16:21:03,...,50.0,2.0,multiple,Nice try! Hint: 4 is a multiple.,2.0,2021-06-22 16:21:12,4.5,9,good,0.0
57,15,2021-06-22 16:20:59,20.0,1.0,easy,1,10,10,4.0,2021-06-22 16:21:03,...,51.0,NaN,None,None,NaN,2021-06-22 16:21:17,4.7,4,None,0.0


In [31]:
games_finished = games_started[games_started.outcome_id.notna()].copy()
games_finished

,session_id,app_start_time,game_id,level_of_difficulty_type_id,level_of_difficulty_type,low_range,high_range,range_size,winning_number,game_start_time,...,guess_error_type,outcome_id,outcome_type_id,outcome_type,score,feedback_type,improvement_area_id,recommendation_type,game_end_time,play_again
0,1,2021-06-22 15:14:16,1.0,1.0,easy,1,10,10,7.0,2021-06-22 15:14:20,...,non_integer,1.0,1.0,win,80.0,None,NaN,None,2021-06-22 15:14:42,1.0
1,1,2021-06-22 15:14:16,1.0,1.0,easy,1,10,10,7.0,2021-06-22 15:14:20,...,NaN,1.0,1.0,win,80.0,None,NaN,None,2021-06-22 15:14:42,1.0
2,1,2021-06-22 15:14:16,1.0,1.0,easy,1,10,10,7.0,2021-06-22 15:14:20,...,out_of_range,1.0,1.0,win,80.0,None,NaN,None,2021-06-22 15:14:42,1.0
3,1,2021-06-22 15:14:16,1.0,1.0,easy,1,10,10,7.0,2021-06-22 15:14:20,...,NaN,1.0,1.0,win,80.0,None,NaN,None,2021-06-22 15:14:42,1.0
4,1,2021-06-22 15:14:16,2.0,1.0,easy,1,10,10,1.0,2021-06-22 15:14:46,...,NaN,2.0,1.0,win,100.0,None,NaN,None,2021-06-22 15:14:51,0.0
5,2,2021-06-22 15:15:02,3.0,2.0,medium,1,100,100,92.0,2021-06-22 15:15:07,...,NaN,3.0,1.0,win,40.0,improvement,1.0,None,2021-06-22 15:15:32,0.0
6,2,2021-06-22 15:15:02,3.0,2.0,medium,1,100,100,92.0,2021-06-22 15:15:07,...,NaN,3.0,1.0,win,40.0,improvement,1.0,None,2021-06-22 15:15:32,0.0
7,2,2021-06-22 15:15:02,3.0,2.0,medium,1,100,100,92.0,2021-06-22 15:15:07,...,NaN,3.0,1.0,win,40.0,improvement,1.0,None,2021-06-22 15:15:32,0.0
8,2,2021-06-22 15:15:02,3.0,2.0,medium,1,100,100,92.0,2021-06-22 15:15:07,...,NaN,3.0,1.0,win,40.0,improvement,1.0,None,2021-06-22 15:15:32,0.0
9,3,2021-06-22 15:15:45,4.0,3.0,hard,1,1000,1000,93.0,2021-06-22 15:15:50,...,NaN,4.0,2.0,lose,NaN,None,NaN,None,2021-06-22 15:16:57,0.0


In [32]:
def calc_won(game_id):
    game_results = games_finished[games_finished.game_id == game_id]
    outcome = game_results.outcome_type_id.unique()[0]
    won = 1 if outcome == 1 else 0
    return won

In [33]:
def calc_game_count(session_id):
    game_ids = list(games_finished.loc[games_finished.session_id == session_id, "game_id"].unique())
    game_count = len(game_ids)
    return game_count

In [34]:
games_finished.insert(33, 'total_duration', games_finished.game_end_time - games_finished.game_start_time)
games_finished['total_duration'] = games_finished.total_duration.dt.total_seconds()
games_finished.total_hints.fillna(0, inplace=True)
games_finished.insert(34, 'won', games_finished.game_id.apply(calc_won))
games_finished.insert(2, 'game_count', games_finished.session_id.apply(calc_game_count))

In [35]:
games_finished

,session_id,app_start_time,game_count,game_id,level_of_difficulty_type_id,level_of_difficulty_type,low_range,high_range,range_size,winning_number,...,outcome_type_id,outcome_type,score,feedback_type,improvement_area_id,recommendation_type,game_end_time,total_duration,won,play_again
0,1,2021-06-22 15:14:16,2,1.0,1.0,easy,1,10,10,7.0,...,1.0,win,80.0,None,NaN,None,2021-06-22 15:14:42,22.0,1,1.0
1,1,2021-06-22 15:14:16,2,1.0,1.0,easy,1,10,10,7.0,...,1.0,win,80.0,None,NaN,None,2021-06-22 15:14:42,22.0,1,1.0
2,1,2021-06-22 15:14:16,2,1.0,1.0,easy,1,10,10,7.0,...,1.0,win,80.0,None,NaN,None,2021-06-22 15:14:42,22.0,1,1.0
3,1,2021-06-22 15:14:16,2,1.0,1.0,easy,1,10,10,7.0,...,1.0,win,80.0,None,NaN,None,2021-06-22 15:14:42,22.0,1,1.0
4,1,2021-06-22 15:14:16,2,2.0,1.0,easy,1,10,10,1.0,...,1.0,win,100.0,None,NaN,None,2021-06-22 15:14:51,5.0,1,0.0
5,2,2021-06-22 15:15:02,1,3.0,2.0,medium,1,100,100,92.0,...,1.0,win,40.0,improvement,1.0,None,2021-06-22 15:15:32,25.0,1,0.0
6,2,2021-06-22 15:15:02,1,3.0,2.0,medium,1,100,100,92.0,...,1.0,win,40.0,improvement,1.0,None,2021-06-22 15:15:32,25.0,1,0.0
7,2,2021-06-22 15:15:02,1,3.0,2.0,medium,1,100,100,92.0,...,1.0,win,40.0,improvement,1.0,None,2021-06-22 15:15:32,25.0,1,0.0
8,2,2021-06-22 15:15:02,1,3.0,2.0,medium,1,100,100,92.0,...,1.0,win,40.0,improvement,1.0,None,2021-06-22 15:15:32,25.0,1,0.0
9,3,2021-06-22 15:15:45,1,4.0,3.0,hard,1,1000,1000,93.0,...,2.0,lose,NaN,None,NaN,None,2021-06-22 15:16:57,67.0,0,0.0


In [36]:
games_summary.insert(20, 'total_duration', games_summary.game_end_time - games_summary.game_start_time)
games_summary['total_duration'] = games_summary.total_duration.dt.total_seconds()
games_summary.insert(21, 'won', games_summary.index.map(lambda x: 1 if games_summary.at[x, 'outcome_type_id'] == 1 else 0))
games_summary.insert(2, 'game_count', games_summary.session_id.apply(lambda x: games_summary.session_id.value_counts()[x]))

In [37]:
games_summary

,session_id,app_start_time,game_count,level_of_difficulty_type_id,level_of_difficulty_type,low_range,high_range,range_size,winning_number,game_start_time,...,outcome_type_id,outcome_type,score,feedback_type,improvement_area_id,recommendation_type,game_end_time,total_duration,won,play_again
game_id,,,,,,,,,,,,,,,,,,,,,
1.0,1,2021-06-22 15:14:16,2,1.0,easy,1,10,10,7.0,2021-06-22 15:14:20,...,1.0,win,80.0,NaN,NaN,NaN,2021-06-22 15:14:42,22.0,1,1.0
2.0,1,2021-06-22 15:14:16,2,1.0,easy,1,10,10,1.0,2021-06-22 15:14:46,...,1.0,win,100.0,NaN,NaN,NaN,2021-06-22 15:14:51,5.0,1,0.0
3.0,2,2021-06-22 15:15:02,1,2.0,medium,1,100,100,92.0,2021-06-22 15:15:07,...,1.0,win,40.0,improvement,1.0,NaN,2021-06-22 15:15:32,25.0,1,0.0
4.0,3,2021-06-22 15:15:45,1,3.0,hard,1,1000,1000,93.0,2021-06-22 15:15:50,...,2.0,lose,NaN,NaN,NaN,NaN,2021-06-22 15:16:57,67.0,0,0.0
7.0,4,2021-06-22 15:17:08,3,4.0,custom,50,75,26,61.0,2021-06-22 15:17:29,...,1.0,win,80.0,NaN,NaN,NaN,2021-06-22 15:17:51,22.0,1,1.0
8.0,4,2021-06-22 15:17:08,3,1.0,easy,1,10,10,1.0,2021-06-22 15:18:02,...,1.0,win,70.0,NaN,NaN,NaN,2021-06-22 15:18:17,15.0,1,1.0
9.0,4,2021-06-22 15:17:08,3,1.0,easy,1,10,10,5.0,2021-06-22 15:18:20,...,1.0,win,90.0,NaN,NaN,NaN,2021-06-22 15:18:27,7.0,1,0.0
10.0,5,2021-06-22 15:18:38,2,2.0,medium,1,100,100,55.0,2021-06-22 15:18:41,...,1.0,win,40.0,NaN,NaN,NaN,2021-06-22 15:19:52,71.0,1,1.0
11.0,5,2021-06-22 15:18:38,2,2.0,medium,1,100,100,5.0,2021-06-22 15:19:58,...,1.0,win,60.0,NaN,NaN,NaN,2021-06-22 15:20:15,17.0,1,0.0


In [38]:
games_summary.iloc[1]

session_id                                       1
app_start_time                 2021-06-22 15:14:16
game_count                                       2
level_of_difficulty_type_id                    1.0
level_of_difficulty_type                      easy
low_range                                        1
high_range                                      10
range_size                                      10
winning_number                                 1.0
game_start_time                2021-06-22 15:14:46
total_guesses                                  1.0
total_hints                                    NaN
guess_errors                                     0
outcome_id                                     2.0
outcome_type_id                                1.0
outcome_type                                   win
score                                        100.0
feedback_type                                  NaN
improvement_area_id                            NaN
recommendation_type            

In [39]:
def calc_hint_power(hint_type_id):
    # Subset games_finished by games using that hint type.
    game_ids = list(games_finished.loc[games_finished.hint_type_id == hint_type_id, "game_id"].unique())
    subset_df = games_finished[games_finished.game_id.isin(game_ids)]

    # Calculate % of those games won
    try:
        games_won = len(list(subset_df.loc[subset_df.outcome_type_id == 1, "game_id"].unique()))
        total_games = len(list(subset_df.game_id.unique()))
        percent_won = round(games_won / total_games * 100, 1)
    except ZeroDivisionError:
        return 0
    
    return percent_won

In [40]:
def rank_hint_types():
    hint_type_ids = list(games_finished.hint_type_id.unique())
    hint_type_ids = [x for x in hint_type_ids if np.isnan(x) == False]
    hint_power = {hint_type_id: calc_hint_power(hint_type_id) for hint_type_id in hint_type_ids}
    hint_rankings = dict(sorted(hint_power.items(), key=lambda x:x[1], reverse=True))

    return hint_rankings

In [41]:
rank_hint_types()

{5.0: 100.0, 3.0: 100.0, 2.0: 91.7, 1.0: 85.7, 6.0: 0.0, 4.0: 0.0}

In [42]:
games_won = games_finished[games_finished.outcome_type_id == 1].copy()
games_won = games_won[games_won.score.notna()].copy()
games_won

,session_id,app_start_time,game_count,game_id,level_of_difficulty_type_id,level_of_difficulty_type,low_range,high_range,range_size,winning_number,...,outcome_type_id,outcome_type,score,feedback_type,improvement_area_id,recommendation_type,game_end_time,total_duration,won,play_again
0,1,2021-06-22 15:14:16,2,1.0,1.0,easy,1,10,10,7.0,...,1.0,win,80.0,None,NaN,None,2021-06-22 15:14:42,22.0,1,1.0
1,1,2021-06-22 15:14:16,2,1.0,1.0,easy,1,10,10,7.0,...,1.0,win,80.0,None,NaN,None,2021-06-22 15:14:42,22.0,1,1.0
2,1,2021-06-22 15:14:16,2,1.0,1.0,easy,1,10,10,7.0,...,1.0,win,80.0,None,NaN,None,2021-06-22 15:14:42,22.0,1,1.0
3,1,2021-06-22 15:14:16,2,1.0,1.0,easy,1,10,10,7.0,...,1.0,win,80.0,None,NaN,None,2021-06-22 15:14:42,22.0,1,1.0
4,1,2021-06-22 15:14:16,2,2.0,1.0,easy,1,10,10,1.0,...,1.0,win,100.0,None,NaN,None,2021-06-22 15:14:51,5.0,1,0.0
5,2,2021-06-22 15:15:02,1,3.0,2.0,medium,1,100,100,92.0,...,1.0,win,40.0,improvement,1.0,None,2021-06-22 15:15:32,25.0,1,0.0
6,2,2021-06-22 15:15:02,1,3.0,2.0,medium,1,100,100,92.0,...,1.0,win,40.0,improvement,1.0,None,2021-06-22 15:15:32,25.0,1,0.0
7,2,2021-06-22 15:15:02,1,3.0,2.0,medium,1,100,100,92.0,...,1.0,win,40.0,improvement,1.0,None,2021-06-22 15:15:32,25.0,1,0.0
8,2,2021-06-22 15:15:02,1,3.0,2.0,medium,1,100,100,92.0,...,1.0,win,40.0,improvement,1.0,None,2021-06-22 15:15:32,25.0,1,0.0
15,4,2021-06-22 15:17:08,3,7.0,4.0,custom,50,75,26,61.0,...,1.0,win,80.0,None,NaN,None,2021-06-22 15:17:51,22.0,1,1.0


In [43]:
def calc_guess_time_ratio(game_id):
    game_results = games_won[games_won.game_id == game_id]
    guess_ids = list(game_results.guess_id.sort_values())
    times_before_guess = []
    
    for guess_id in guess_ids:
        guess_time = game_results.loc[game_results.guess_id == guess_id, "guess_entry_time"]
        
        if not times_before_guess:
            time = guess_time - game_results.game_start_time.iloc[0]
        else:
            previous_guess_time = game_results.loc[game_results.guess_id == guess_id - 1, "guess_entry_time"].iloc[0]
            time = guess_time - previous_guess_time
        
        time = time.dt.total_seconds()
        time = time.iat[0]
        times_before_guess.append(time)
    
    guess_time_ratio = round(max(times_before_guess) / min(times_before_guess), 2)
    
    return guess_time_ratio

In [44]:
games_won.insert(34, 'guess_time_ratio', games_won.game_id.apply(calc_guess_time_ratio))

In [45]:
games_won

,session_id,app_start_time,game_count,game_id,level_of_difficulty_type_id,level_of_difficulty_type,low_range,high_range,range_size,winning_number,...,outcome_type,score,feedback_type,improvement_area_id,recommendation_type,game_end_time,guess_time_ratio,total_duration,won,play_again
0,1,2021-06-22 15:14:16,2,1.0,1.0,easy,1,10,10,7.0,...,win,80.0,None,NaN,None,2021-06-22 15:14:42,1.20,22.0,1,1.0
1,1,2021-06-22 15:14:16,2,1.0,1.0,easy,1,10,10,7.0,...,win,80.0,None,NaN,None,2021-06-22 15:14:42,1.20,22.0,1,1.0
2,1,2021-06-22 15:14:16,2,1.0,1.0,easy,1,10,10,7.0,...,win,80.0,None,NaN,None,2021-06-22 15:14:42,1.20,22.0,1,1.0
3,1,2021-06-22 15:14:16,2,1.0,1.0,easy,1,10,10,7.0,...,win,80.0,None,NaN,None,2021-06-22 15:14:42,1.20,22.0,1,1.0
4,1,2021-06-22 15:14:16,2,2.0,1.0,easy,1,10,10,1.0,...,win,100.0,None,NaN,None,2021-06-22 15:14:51,1.00,5.0,1,0.0
5,2,2021-06-22 15:15:02,1,3.0,2.0,medium,1,100,100,92.0,...,win,40.0,improvement,1.0,None,2021-06-22 15:15:32,3.00,25.0,1,0.0
6,2,2021-06-22 15:15:02,1,3.0,2.0,medium,1,100,100,92.0,...,win,40.0,improvement,1.0,None,2021-06-22 15:15:32,3.00,25.0,1,0.0
7,2,2021-06-22 15:15:02,1,3.0,2.0,medium,1,100,100,92.0,...,win,40.0,improvement,1.0,None,2021-06-22 15:15:32,3.00,25.0,1,0.0
8,2,2021-06-22 15:15:02,1,3.0,2.0,medium,1,100,100,92.0,...,win,40.0,improvement,1.0,None,2021-06-22 15:15:32,3.00,25.0,1,0.0
15,4,2021-06-22 15:17:08,3,7.0,4.0,custom,50,75,26,61.0,...,win,80.0,None,NaN,None,2021-06-22 15:17:51,2.20,22.0,1,1.0


## Predictions

### Score

In [46]:
from sklearn import linear_model
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score

In [47]:
class Model:
    """
    This class is for building a model, training it, and using it to make predictions.
    
    Attributes:
        model_type: The type of model used to make predictions.  This is provided when instantiating the object.
        data (dataframe): The pandas dataframe used to train the model and use it for predictions.
        features (list): The variables used by the model to make predictions.
        target: The value the model aims to predict.
        model: The model after it has been fitted to the training data.
    """
    
    def __init__(self, model, data, features, target, model_type, model_name):
        """This method is the constructor for the model class."""
        self.model = model
        self.data = data
        self.features = features
        self.target = target
        self.type = model_type
        self.name = model_name
        self.predicted_target = None
        self.observed_target = None
        self.residuals = None
    
    def build_model(self, compare_df, test_size=.2):
        """This method splits the dataframe into training and test sets, trains the model on the training data, 
        makes predictions on the test set, and saves metrics in the compare_df dataframe."""
        
        # Split data set into training and test sets based on the test_size parameter
        features_train, features_test, target_train, target_test = train_test_split(
            self.data[self.features], self.data[self.target], test_size=test_size)
        
        # Fit the model to the training data and save the score as a variable
        self.model = self.model.fit(features_train, target_train)
        training_score = self.model.score(features_train, target_train)
        
        # Make predictions on the test set and save the R^2 and MSE scores as variables
        self.predicted_target = self.model.predict(features_test)
        self.observed_target = target_test
        self.residuals = target_test - self.predicted_target
        
        if self.type == 'Regression':
            test_r2 = r2_score(y_true=target_test, y_pred=self.predicted_target)
            mse = mean_squared_error(y_true=target_test, y_pred=self.predicted_target)
            rmse = mse**.5
            metrics = [training_score, test_r2, rmse, 0]
        elif self.type == 'Classification':
            accuracy = accuracy_score(y_true=target_test, y_pred=self.predicted_target)
            precision = precision_score(y_true=target_test, y_pred=self.predicted_target)
            recall = recall_score(y_true=target_test, y_pred=self.predicted_target)
            metrics = [training_score, accuracy, precision, recall]
        
        # Add the 3 scores to the compare_df (given as an argument)
        compare_df.loc[self.name] = metrics
        
        return self.model
    
    def cross_validate(self, cv, compare_df, scoring='neg_mean_squared_error'):
        """This method runs cross validation on the dataset."""
        neg_mse = cross_val_score(self.model, self.data[self.features], self.data[self.target],  cv=cv, scoring=scoring)
        avg_mse = sum(neg_mse) / len(neg_mse) * -1.0
        avg_rmse = avg_mse**.5
        compare_df.loc[self.name, 'Cross Validation Score'] = avg_rmse
    
    def get_coefficients(self, test_size=.2):
        """This method gets the coefficients of the model and displays them as a series."""
        features_train, features_test, target_train, target_test = train_test_split(
            self.data[self.features], self.data[self.target], test_size=test_size)
        self.model = self.model.fit(features_train, target_train)
        coefficients = pd.Series(self.model.coef_, index=features_train.columns).sort_values(ascending=False)
        return coefficients
    
    def plot_observed_vs_fitted(self):
        """This method creates a line graph for the predicted values and the observed values to visualize how well the model
        predicts the target variable."""
        plt.plot(self.predicted_target, c='g')
        plt.plot(self.observed_target, c='b')
        plt.title('Observed vs Fitted Graph')
    
    def predict(self, data):
        """This method makes predictions on the given dataset."""
        predictions = self.model.predict(data)
        return predictions

In [48]:
score_pred_features = [
    "range_size",
    "avg_guess_time",
    "total_hints",
    "total_duration",
    "guess_time_ratio"
]
score_pred_target = "score"

lreg = linear_model.LinearRegression()

score_model_comparison = pd.DataFrame(columns=['Training R2', 'Test R2', 'Test RMSE', 'Cross Validation Score'])

In [49]:
score_predict_model = Model(lreg, games_won, score_pred_features, score_pred_target, "Regression", "Linear Reg - Score - All")
score_predict_model.build_model(score_model_comparison)
score_predict_model.cross_validate(4, score_model_comparison)

for feature in score_pred_features:
    model = Model(lreg, games_won, [feature], score_pred_target, "Regression", f"Linear Reg - Score - {feature}")
    model.build_model(score_model_comparison)
    model.cross_validate(4, score_model_comparison)

score_predict_model2 = Model(lreg, games_won, ["total_hints","total_duration"], score_pred_target, "Regression", "Linear Reg - Score - Custom2")
score_predict_model2.build_model(score_model_comparison)
score_predict_model2.cross_validate(4, score_model_comparison)

score_predict_model3 = Model(lreg, games_won, ["total_hints","total_duration","guess_time_ratio"], score_pred_target, "Regression", "Linear Reg - Score - Custom3")
score_predict_model3.build_model(score_model_comparison)
score_predict_model3.cross_validate(4, score_model_comparison)

score_predict_model4 = Model(lreg, games_won, ["range_size","avg_guess_time"], score_pred_target, "Regression", "Linear Reg - Score - Custom4")
score_predict_model4.build_model(score_model_comparison)
score_predict_model4.cross_validate(4, score_model_comparison)

score_model_comparison

,Training R2,Test R2,Test RMSE,Cross Validation Score
Linear Reg - Score - All,0.736502,0.297707,13.102353,82.423382
Linear Reg - Score - range_size,0.174604,0.192948,15.719328,113.637369
Linear Reg - Score - avg_guess_time,0.002284,-0.544536,23.997168,19.411360
Linear Reg - Score - total_hints,0.575410,0.504951,13.450157,12.918192
Linear Reg - Score - total_duration,0.296877,-0.239196,19.634861,20.351360
Linear Reg - Score - guess_time_ratio,0.222888,-0.115944,18.706632,20.866587
Linear Reg - Score - Custom2,0.584042,0.645909,10.864176,14.269791
Linear Reg - Score - Custom3,0.676717,0.125554,14.912744,15.517489
Linear Reg - Score - Custom4,0.219666,-0.017557,20.606071,118.514013


In [50]:
score_predict_model.get_coefficients()

avg_guess_time       0.300521
guess_time_ratio     0.247235
range_size          -0.028665
total_duration      -0.063824
total_hints        -17.454888
dtype: float64

### Outcome

In [51]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

In [52]:
outcome_pred_features = [
    "range_size",
    "avg_guess_time",
    "total_hints",
    "total_duration"
]
outcome_pred_target = "won"

log_reg = linear_model.LogisticRegression(C=1)

outcome_model_comparison = pd.DataFrame(columns=['Training Accuracy', 'Test Accuracy', 'Test Precision', 'Test Recall'])

In [53]:
outcome_predict_model = Model(log_reg, games_finished, outcome_pred_features, outcome_pred_target, "Classification", "Logistic Reg - Outcome - All")
outcome_predict_model.build_model(outcome_model_comparison)

for feature in outcome_pred_features:
    model = Model(log_reg, games_finished, [feature], outcome_pred_target, "Classification", f"Logistic Reg - Outcome - {feature}")
    model.build_model(outcome_model_comparison)

score_predict_model2 = Model(log_reg, games_finished, ["range_size","avg_guess_time"], outcome_pred_target, "Classification", "Logistic Reg - Outcome - Custom2")
score_predict_model2.build_model(outcome_model_comparison)

# score_predict_model3 = Model(lreg, games_won, ["total_hints","total_duration","guess_time_ratio"], score_pred_target, "Regression", "Linear Reg - Score - Custom3")
# score_predict_model3.build_model(score_model_comparison)
# score_predict_model3.cross_validate(4, score_model_comparison)

outcome_model_comparison

,Training Accuracy,Test Accuracy,Test Precision,Test Recall
Logistic Reg - Outcome - All,1.000000,0.8,1.0,0.8
Logistic Reg - Outcome - range_size,0.944444,0.9,1.0,0.9
Logistic Reg - Outcome - avg_guess_time,0.888889,1.0,1.0,1.0
Logistic Reg - Outcome - total_hints,1.000000,1.0,1.0,1.0
Logistic Reg - Outcome - total_duration,0.916667,0.9,0.9,1.0
Logistic Reg - Outcome - Custom2,0.944444,0.8,0.8,1.0


## Save Data

In [54]:
game_data.to_csv("data/game_data.csv", index=False)
no_games.to_csv("data/no_games.csv", index=False)
range_errors.to_csv("data/range_errors.csv", index=False)
games_summary.to_csv("data/games_summary.csv", index=False)
games_not_finished.to_csv("data/games_not_finished.csv", index=False)
games_finished.to_csv("data/games_finished.csv", index=False)
games_won.to_csv("data/games_won.csv", index=False)